In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Consumo de Cimento (t)
0,2003,0.760059,3.282770,8.177444,1197.520265,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,635.153000
1,2004,0.762013,3.237874,8.107574,1241.084984,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,6.442102e+07,3.565093e+06,16.896386,5.992647e+07,636.659000
2,2005,0.763817,3.103762,8.084884,1360.910937,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,6.528363e+07,3.601258e+06,16.957151,6.021671e+07,768.967000
3,2006,0.765477,3.130953,7.998040,1619.896813,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,6.602371e+07,3.639069e+06,16.990689,6.039472e+07,673.561000
4,2007,0.766998,3.335837,7.969140,1900.181823,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,6.663355e+07,3.646642e+06,16.999308,6.046495e+07,974.754000
5,2008,0.768383,3.594891,7.934578,2065.641254,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,1263.713000
6,2009,0.769638,3.774692,7.964300,2200.127740,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,1138.824000
7,2010,0.770767,3.930070,7.927576,2376.087362,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,1223.466000
8,2011,0.771775,4.104068,7.917310,2497.788723,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,1271.961000
9,2012,0.772417,4.250916,9.395868,2521.227215,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,1354.740000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado
0,0.760059,3.282770,8.177444,1197.520265,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,6.342537e+07,3.523987e+06,16.799308,5.948249e+07
1,0.762013,3.237874,8.107574,1241.084984,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,6.442102e+07,3.565093e+06,16.896386,5.992647e+07
2,0.763817,3.103762,8.084884,1360.910937,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,6.528363e+07,3.601258e+06,16.957151,6.021671e+07
3,0.765477,3.130953,7.998040,1619.896813,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,6.602371e+07,3.639069e+06,16.990689,6.039472e+07
4,0.766998,3.335837,7.969140,1900.181823,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,6.663355e+07,3.646642e+06,16.999308,6.046495e+07
5,0.768383,3.594891,7.934578,2065.641254,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,6.711793e+07,3.666068e+06,16.994002,6.044984e+07
6,0.769638,3.774692,7.964300,2200.127740,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,6.738295e+07,3.683277e+06,16.935445,6.029388e+07
7,0.770767,3.930070,7.927576,2376.087362,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,6.901541e+07,3.673390e+06,16.970783,6.069300e+07
8,0.771775,4.104068,7.917310,2497.788723,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,7.032342e+07,3.655681e+06,16.962954,6.083140e+07
9,0.772417,4.250916,9.395868,2521.227215,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,7.118644e+07,3.633845e+06,16.845395,6.065703e+07


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      636.659000
1      768.967000
2      673.561000
3      974.754000
4     1263.713000
5     1138.824000
6     1223.466000
7     1271.961000
8     1354.740000
9     1378.303000
10    1296.041857
11    1250.323886
12    1210.917257
13    1097.369000
14    1146.390000
15    1139.906000
16    1138.655000
17    1135.022000
18    1040.260000
19            NaN
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado
0,-1.890494,-1.225902,-0.764918,-1.095771,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.833535,-0.888456,0.123092,-1.631909
1,-1.556140,-1.310067,-0.822986,-1.014041,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.520319,-0.282403,0.596678,-0.539076
2,-1.247300,-1.561483,-0.841843,-0.789240,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.248956,0.250792,0.893115,0.175364
3,-0.963178,-1.510509,-0.914016,-0.303366,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.016136,0.808255,1.056729,0.613532
4,-0.702973,-1.126418,-0.938034,0.222467,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.824291,0.919918,1.098773,0.786398
5,-0.465887,-0.640777,-0.966757,0.532880,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.671911,1.206320,1.072888,0.749189
6,-0.251122,-0.303710,-0.942056,0.785185,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.588541,1.460043,0.787225,0.365309
7,-0.057878,-0.012427,-0.972576,1.115296,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.074993,1.314280,0.959620,1.347726
8,0.114644,0.313763,-0.981108,1.343616,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.336487,1.053173,0.921424,1.688405
9,0.224627,0.589055,0.247678,1.387588,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.607980,0.731237,0.347923,1.259187


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      636.659000
1      768.967000
2      673.561000
3      974.754000
4     1263.713000
5     1138.824000
6     1223.466000
7     1271.961000
8     1354.740000
9     1378.303000
10    1296.041857
11    1250.323886
12    1210.917257
13    1097.369000
14    1146.390000
15    1139.906000
16    1138.655000
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado
17,0.910005,2.096358,0.751123,-1.604501,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.700733,-0.492713,-1.07591,-1.001567


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1135.022
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [15]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4150831992, 171414464, 3627798560, 1515124794, 3049688925, 120687601, 444124577, 247076742, 263373595, 1840683268, 2732122938, 2451859048, 2005231055, 3748595147, 3261229055, 1954687654, 3634026910, 2912517699, 3110007648, 1833972601, 4106535368, 871413509, 1565712507, 798460869, 3662033289, 3790565668, 885808194, 1935110277, 340163008, 1635392271, 3535148429, 1307432964, 1396394952, 2650920282, 1325351962, 4235806727, 4203761896, 4058100013, 2290784799, 920584479, 1958725867, 41931631, 3205788980, 2911097115, 179751106, 2731581955, 2828352381, 888987938, 1353874009, 3085769635, 3669670137, 3057533467, 2394685927, 2572946250, 384389505, 3730831203, 1885761579, 2347568359, 2528843369, 833279054, 3405728902, 1701704007, 368882241, 3198708117, 4139774570, 2712015093, 4069181175, 51961772, 3431773723, 2507367422, 500488920, 2779219660, 3986758921, 709155139, 3822454112, 27034471, 463240835, 3881445276, 3339199104, 3366539965, 3283475077, 709604415, 1578147459, 3369977171, 1490583449, 1227

2023-10-01 11:55:47.656120: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 11.493046760559082


Step: 24 ___________________________________________
val_loss: 11.538739204406738


Step: 25 ___________________________________________
val_loss: 18.208335876464844


Step: 26 ___________________________________________
val_loss: 12.737509727478027


Step: 27 ___________________________________________
val_loss: 9.64637279510498
winner_seed: 1935110277


Step: 28 ___________________________________________
val_loss: 8652.3359375


Step: 29 ___________________________________________
val_loss: 156.03456115722656


Step: 30 ___________________________________________
val_loss: 4.724075794219971
winner_seed: 3535148429


Step: 31 ___________________________________________
val_loss: 829.5726928710938


Step: 32 ___________________________________________
val_loss: 11.4941987991333


Step: 33 ___________________________________________
val_loss: 17071.615234375


Step: 34 ___________________________________________
val_loss: 498.8108825683594


Step: 35 ____

2023-10-01 12:02:08.826808: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 11.516716003417969


Step: 48 ___________________________________________
val_loss: 7.175453186035156


Step: 49 ___________________________________________
val_loss: 11.494281768798828


Step: 50 ___________________________________________
val_loss: 838.0087280273438


Step: 51 ___________________________________________
val_loss: 2094.990478515625


Step: 52 ___________________________________________
val_loss: 11.54794692993164


Step: 53 ___________________________________________
val_loss: 11.493786811828613


Step: 54 ___________________________________________
val_loss: 11.4932222366333


Step: 55 ___________________________________________
val_loss: 11.495902061462402


Step: 56 ___________________________________________
val_loss: 11.689349174499512


Step: 57 ___________________________________________
val_loss: 12.047203063964844


Step: 58 ___________________________________________
val_loss: 11.532028198242188


Step: 59 __________________________________________

In [16]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 450ms/step - loss: 1292407.7500 - val_loss: 1277686.1250
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 1261110.1250 - val_loss: 1721121.3750
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 1344031.2500 - val_loss: 769721.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 1344789.6250 - val_loss: 1234265.8750
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 1101598.6250 - val_loss: 1325008.8750
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 1066133.2500 - val_loss: 1174412.3750
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 1042203.7500 - val_loss: 1218311.6250
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 983251.9375 - val_loss: 1120217.6250
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 956284.3125 - val_loss: 1286951

1/1 [==============================] - 0s 14ms/step - loss: 254236.9062 - val_loss: 277712.0938
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 228727.2969 - val_loss: 326285.5000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 248935.3750 - val_loss: 254057.9219
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 262679.9375 - val_loss: 243651.5469
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 219310.7188 - val_loss: 214891.5000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 214921.4062 - val_loss: 256781.5781
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 256787.3594 - val_loss: 302927.0312
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 213881.5000 - val_loss: 327119.0312
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 201520.2031 - val_loss: 198968.3594
Epoch 83/10000
1

1/1 [==============================] - 0s 13ms/step - loss: 113052.2812 - val_loss: 122625.4141
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 106052.1875 - val_loss: 97372.4922
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 118034.6875 - val_loss: 103702.2578
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 93382.7109 - val_loss: 108240.1016
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 106005.8125 - val_loss: 64140.4062
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 102288.0469 - val_loss: 82453.6641
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 92997.6250 - val_loss: 86105.9375
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 109134.1016 - val_loss: 112984.5938
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 106261.1484 - val_loss: 95520.0859
Epoch 156/10000

1/1 [==============================] - 0s 14ms/step - loss: 42742.4375 - val_loss: 35680.8633
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 42193.6484 - val_loss: 35381.7617
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 42164.8516 - val_loss: 33507.6914
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 40819.0898 - val_loss: 35084.8320
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 41057.6719 - val_loss: 31262.0098
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 39725.7070 - val_loss: 29914.9121
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 40244.0898 - val_loss: 33562.7891
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 39273.5312 - val_loss: 28632.8926
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 37479.2070 - val_loss: 28985.4863
Epoch 230/10000
1/1 [=====

Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 19592.3945 - val_loss: 3290.0012
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 19232.5430 - val_loss: 4763.7988
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 20075.5234 - val_loss: 3130.8582
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 19761.5391 - val_loss: 3243.5051
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 19248.6543 - val_loss: 3298.7581
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 18154.2617 - val_loss: 2662.9719
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 17947.9531 - val_loss: 2644.6995
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 19896.4219 - val_loss: 3174.8782
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 19104.7617 - val_loss: 2884.4199
Epoch 305/10000
1/1

...

Epoch 749/10000
1/1 [==============================] - 0s 13ms/step - loss: 20981.8906 - val_loss: 15.9151
Epoch 750/10000
1/1 [==============================] - 0s 13ms/step - loss: 17009.1191 - val_loss: 268.4366
Epoch 751/10000
1/1 [==============================] - 0s 14ms/step - loss: 16652.9277 - val_loss: 193.1777
Epoch 752/10000
1/1 [==============================] - 0s 14ms/step - loss: 16756.4980 - val_loss: 207.4703
Epoch 753/10000
1/1 [==============================] - 0s 13ms/step - loss: 17804.6758 - val_loss: 306.3007
Epoch 754/10000
1/1 [==============================] - 0s 13ms/step - loss: 16687.8242 - val_loss: 182.5762
Epoch 755/10000
1/1 [==============================] - 0s 13ms/step - loss: 16968.8281 - val_loss: 210.9236
Epoch 756/10000
1/1 [==============================] - 0s 13ms/step - loss: 19770.7109 - val_loss: 33.6660
Epoch 757/10000
1/1 [==============================] - 0s 13ms/step - loss: 23437.5820 - val_loss: 423.4446
Epoch 758/10000
1/1 [=========

1/1 [==============================] - 0s 13ms/step - loss: 17677.8086 - val_loss: 589.4655
Epoch 826/10000
1/1 [==============================] - 0s 13ms/step - loss: 21041.6855 - val_loss: 11.7396
Epoch 827/10000
1/1 [==============================] - 0s 14ms/step - loss: 17996.7266 - val_loss: 492.8076
Epoch 828/10000
1/1 [==============================] - 0s 14ms/step - loss: 25593.0273 - val_loss: 11.8277
Epoch 829/10000
1/1 [==============================] - 0s 13ms/step - loss: 17394.1934 - val_loss: 657.6799
Epoch 830/10000
1/1 [==============================] - 0s 13ms/step - loss: 17783.4785 - val_loss: 497.1559
Epoch 831/10000
1/1 [==============================] - 0s 13ms/step - loss: 16980.1406 - val_loss: 540.3714
Epoch 832/10000
1/1 [==============================] - 0s 14ms/step - loss: 21130.1660 - val_loss: 13.3611
Epoch 833/10000
1/1 [==============================] - 0s 13ms/step - loss: 20080.3477 - val_loss: 69.1995
Epoch 834/10000
1/1 [===========================

1/1 [==============================] - 0s 13ms/step - loss: 17444.2480 - val_loss: 304.5343
Epoch 902/10000
1/1 [==============================] - 0s 14ms/step - loss: 17217.8594 - val_loss: 230.2106
Epoch 903/10000
1/1 [==============================] - 0s 13ms/step - loss: 17621.7734 - val_loss: 395.6011
Epoch 904/10000
1/1 [==============================] - 0s 13ms/step - loss: 17226.5527 - val_loss: 214.6054
Epoch 905/10000
1/1 [==============================] - 0s 13ms/step - loss: 17202.5156 - val_loss: 235.4810
Epoch 906/10000
1/1 [==============================] - 0s 14ms/step - loss: 17193.8496 - val_loss: 230.8357
Epoch 907/10000
1/1 [==============================] - 0s 13ms/step - loss: 17270.2070 - val_loss: 237.6309
Epoch 908/10000
1/1 [==============================] - 0s 14ms/step - loss: 16692.2461 - val_loss: 584.5690
Epoch 909/10000
1/1 [==============================] - 0s 13ms/step - loss: 17263.7207 - val_loss: 187.2961
Epoch 910/10000
1/1 [=======================

Epoch 977/10000
1/1 [==============================] - 0s 14ms/step - loss: 16541.1289 - val_loss: 1533.7438
Epoch 978/10000
1/1 [==============================] - 0s 14ms/step - loss: 17076.9902 - val_loss: 91.8031
Epoch 979/10000
1/1 [==============================] - 0s 13ms/step - loss: 16511.0176 - val_loss: 323.2333
Epoch 980/10000
1/1 [==============================] - 0s 13ms/step - loss: 16217.3711 - val_loss: 958.5150
Epoch 981/10000
1/1 [==============================] - 0s 13ms/step - loss: 16635.8613 - val_loss: 219.8088
Epoch 982/10000
1/1 [==============================] - 0s 13ms/step - loss: 16931.7070 - val_loss: 229.5828
Epoch 983/10000
1/1 [==============================] - 0s 14ms/step - loss: 15777.1963 - val_loss: 1097.3156
Epoch 984/10000
1/1 [==============================] - 0s 13ms/step - loss: 17445.2441 - val_loss: 95.3463
Epoch 985/10000
1/1 [==============================] - 0s 13ms/step - loss: 17017.1758 - val_loss: 73.1369
Epoch 986/10000
1/1 [========

1/1 [==============================] - 0s 14ms/step - loss: 14789.2822 - val_loss: 1061.9640
Epoch 1053/10000
1/1 [==============================] - 0s 14ms/step - loss: 12158.4844 - val_loss: 146.9508
Epoch 1054/10000
1/1 [==============================] - 0s 13ms/step - loss: 11820.6641 - val_loss: 216.6332
Epoch 1055/10000
1/1 [==============================] - 0s 13ms/step - loss: 14657.3184 - val_loss: 59.6434
Epoch 1056/10000
1/1 [==============================] - 0s 14ms/step - loss: 23141.4297 - val_loss: 3366.2908
Epoch 1057/10000
1/1 [==============================] - 0s 14ms/step - loss: 20852.2832 - val_loss: 525.7819
Epoch 1058/10000
1/1 [==============================] - 0s 14ms/step - loss: 11765.9482 - val_loss: 184.7106
Epoch 1059/10000
1/1 [==============================] - 0s 14ms/step - loss: 11609.6846 - val_loss: 221.5437
Epoch 1060/10000
1/1 [==============================] - 0s 13ms/step - loss: 19194.7969 - val_loss: 1303.6959
Epoch 1061/10000
1/1 [============

In [17]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1135.022](test_target) - [[1141.7998]](prediction) = 6.7778046875000655


In [18]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [19]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado
0,-1.660580,-0.491540,1.833233,-1.286771,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.647839,-1.904113,-2.273856,-2.143985
1,-1.104592,-0.647955,1.017393,-1.184412,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.048525,-1.128084,-0.736403,-0.887827
2,-0.591032,-1.115197,0.752453,-0.902872,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.529294,-0.445347,0.225948,-0.066614
3,-0.118573,-1.020465,-0.261576,-0.294365,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.083809,0.268464,0.757106,0.437039
4,0.314114,-0.306654,-0.599026,0.364185,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.283271,0.411445,0.893598,0.635740
5,0.708357,0.595880,-1.002589,0.752944,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.574838,0.778171,0.809566,0.592970
6,1.065484,1.222300,-0.655538,1.068930,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.734361,1.103054,-0.117810,0.151718
7,1.386823,1.763633,-1.084349,1.482361,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.716996,0.916410,0.441852,1.280959


train_target:


,Espírito Santo - Consumo de Cimento (t)
0,636.659
1,768.967
2,673.561
3,974.754
4,1263.713
5,1138.824
6,1223.466
7,1271.961


test_input:


...

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado
17,0.910005,2.096358,0.751123,-1.604501,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.700733,-0.492713,-1.07591,-1.001567


test_target:


,Espírito Santo - Consumo de Cimento (t)
17,1135.022


1/1 [==============================] - 0s 34ms/step
Error: 6.7778046875000655




[1174.541259765625,
 1202.41162109375,
 1338.2196044921875,
 1271.953125,
 1243.52490234375,
 1230.3441162109375,
 1206.4586181640625,
 1151.441650390625,
 1126.678955078125,
 1141.7998046875]

In [20]:
display(targets)
display(predictions)

[1354.74,
 1378.303,
 1296.041857142857,
 1250.3238857142856,
 1210.9172571428571,
 1097.369,
 1146.39,
 1139.906,
 1138.655,
 1135.022]

[1174.541259765625,
 1202.41162109375,
 1338.2196044921875,
 1271.953125,
 1243.52490234375,
 1230.3441162109375,
 1206.4586181640625,
 1151.441650390625,
 1126.678955078125,
 1141.7998046875]

In [21]:
mae = mean_absolute_error(predictions, targets)
mae

67.5837985351563

In [22]:
porcentage = mae/np.mean(targets)
porcentage

0.05563520383925236